In [1]:
import numpy as np

### Setup Sudoku Board

In [2]:
board = np.empty((9, 9))
board[:,:] = np.nan

board = np.array(
[
    [1, 5, 4,   0, 0, 6,   2, 7, 0],
    [0, 0, 8,   1, 7, 0,   0, 4, 5],
    [0, 0, 6,   4, 9, 0,   0, 1, 0],

    [0, 0, 9,   6, 8, 3,   0, 0, 0],
    [3, 2, 1,   0, 0, 0,   8, 6, 0],
    [6, 0, 0,   0, 0, 1,   9, 3, 0],

    [5, 0, 0,   9, 0, 0,   0, 0, 3],
    [0, 9, 7,   0, 0, 0,   1, 0, 2],
    [8, 0, 0,   0, 5, 4,   0, 0, 0],
])

### Utility functions

In [3]:
def possible_values(board, row, col):
    """ find all possible value for (row, col), given the current state of the whole board """
    
    # assume all numbers 1-9 are possible
    vals = range(1, 10)
    
    # exclude row vals
    vals = np.setdiff1d(vals, board[row, :])
    
    # exclude columnn vals
    vals = np.setdiff1d(vals, board[:, col])

    # exclude box vals
    box_row = 3 * (row // 3)
    box_col = 3 * (col // 3)
    vals = np.setdiff1d(vals, board[box_row:box_row + 3, box_col:box_col + 3])

    return vals

def next_open_square(board):
    """ find the next unfilled square on the board """
    
    # if no open squares
    if np.min(board) != 0:
        return None
    
    # find cell of first zero
    next_open = np.argmin(board)

    # need to compute row and column, because argmin flattens the array into 1 dimension
    return (next_open // 9, next_open % 9)


### Solver

In [4]:
def solve(board):
    
    # find an open square to try to fill
    next_open = next_open_square(board)
    
    # if all squares are filled
    if next_open == None:
        # puzzle is completed
        return True, board
    else:
        row, col = next_open
    
    pv = possible_values(board, row, col)
    
    # if no values are possible, then we've failed to find a solution
    if pv.shape[0] == 0:
        return False, None

    # if only one value is possible, 
    elif pv.shape[0] == 1:
        
        # use it
        board[row, col] = pv[0]
        
        # try to solve the rest of the board
        return solve(board.copy())
    
    # if multiple values are possible
    else:

        # for each possible value
        for val in possible_values(board, row, col):
            
            # apply candidate value
            board[row, col] = val

            # try to solve the rest of the board
            outcome, solved_board = solve(board.copy())
            
            # if a solution was found
            if outcome == True:
                return outcome, solved_board
            
        # if here, no solution was found
        return False, None

In [5]:
outcome, solution = solve(board)

In [6]:
solution

array([[1, 5, 4, 8, 3, 6, 2, 7, 9],
       [9, 3, 8, 1, 7, 2, 6, 4, 5],
       [2, 7, 6, 4, 9, 5, 3, 1, 8],
       [7, 4, 9, 6, 8, 3, 5, 2, 1],
       [3, 2, 1, 5, 4, 9, 8, 6, 7],
       [6, 8, 5, 7, 2, 1, 9, 3, 4],
       [5, 6, 2, 9, 1, 7, 4, 8, 3],
       [4, 9, 7, 3, 6, 8, 1, 5, 2],
       [8, 1, 3, 2, 5, 4, 7, 9, 6]])

### Make sure 

In [7]:
def check_sudoku_solution(board):
    
    valid = True
    
    for i in range(9):
        valid &= np.equal(sorted(board[i, :]), range(1, 10)).all()
        valid &= np.equal(sorted(board[:, i]), range(1, 10)).all()
        
        box_x = 3 * (i // 3)
        box_y = 3 * (i % 3)
        valid &= np.equal(sorted(board[box_x:box_x+3, box_y:box_y+3].reshape(-1)), range(1, 10)).all()
        
    return valid

check_sudoku_solution(solution)

True